<a href="https://colab.research.google.com/github/BJHUN/Big-Data/blob/main/Level_1_Amazon_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-12 02:12:33--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-05-12 02:12:34 (9.87 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
# spark.driver.extraClassPath=/home/hadoop/postgresql-42.2.18.jar 
# spark.jars.packages=org.postgresql:postgresql:42.2.18
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData").config("spark.driver.extraClassPath=/home/hadoop/postgresql-42.2.16.jar","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame (extract)
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
# Count the number of records (rows) in the dataset.
df.count()

1785997

In [6]:
#  Create the review_id_df
# CREATE TABLE review_id_table (
#   review_id TEXT PRIMARY KEY NOT NULL,
#   customer_id INTEGER,
#   product_id TEXT,
#   product_parent INTEGER,
#   review_date DATE -- this should be in the formate yyyy-mm-dd
# );

# View df schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# transform review_date into Date field
from pyspark.sql.functions import to_date

# create review_id_df
review_id_df = df.select(['review_id','customer_id','product_id','product_parent', to_date('review_date', 'yyyy-mm-dd')]).drop_duplicates(['review_id'])
review_id_df.show()

+--------------+-----------+----------+--------------+--------------------------------+
|     review_id|customer_id|product_id|product_parent|to_date(review_date, yyyy-mm-dd)|
+--------------+-----------+----------+--------------+--------------------------------+
|R10003I619LWL0|    4317219|B001REZLY8|     816407837|                      2015-01-10|
|R100078OO83YQB|   31850030|B0016HM45K|     670563392|                      2008-01-29|
|R10009XH2FY9Q0|   51098385|B00000IOQV|     845394808|                      1999-01-28|
|R1000EJULTHQ16|   12211018|B002I0H7K6|      22716581|                      2012-01-05|
|R1000GECIM9DZG|   10430321|B006SVUQNQ|     523228689|                      2014-01-06|
|R1000ID99V4TKO|    4971033|B00KY1HZ80|     886096191|                      2015-01-18|
|R1000U9GM6NPGS|   13626862|B00DUJKIO6|     518264079|                      2014-01-23|
|R1000WZPINB01L|   43132564|B0041HROGG|     166509481|                      2012-01-07|
|R1000XAVJH7WUV|   25431956|B000

In [8]:
#  Create the review_id_df
# CREATE TABLE review_id_table (
#   review_id TEXT PRIMARY KEY NOT NULL,
#   customer_id INTEGER,
#   product_id TEXT,
#   product_parent INTEGER,
#   review_date DATE -- this should be in the formate yyyy-mm-dd
# );
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- to_date(review_date, yyyy-mm-dd): date (nullable = true)



In [17]:
# create customers_df
# -- Customer table for first data set
# CREATE TABLE customers (
#   customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#   customer_count INT
# );

customers_df = df.groupby('customer_id').agg({'customer_id':'count'})
customers_df = customers_df.withColumnRenamed('count(customer_id','customer_count')
customers_df.show()


+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   48670265|                 1|
|   49103216|                 2|
|    1131200|                 1|
|   43076447|                 2|
|   46261368|                 1|
|    4883305|                 5|
|   41192649|                 1|
|   40985731|                 7|
|   10437900|                 2|
|   22245671|                 1|
|    2574873|                 1|
|    4696154|                 1|
|    5621202|                 1|
|    5871933|                 2|
|   44089812|                 1|
|    2845910|                 1|
|    5274369|                 1|
|   39069693|                 2|
|     137793|                 1|
|   31914942|                 3|
+-----------+------------------+
only showing top 20 rows



In [10]:
# Load into AWS RDS instance
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://database-1.cmgmbxtsdxem.us-west-1.rds.amazonaws.com:5432/video_review_db"
config = {"user":"postgres",
          "password": "postgres",
          "driver":"org.postgresql.Driver"}

In [11]:
# Write the id_reivew_db to df
# employee_personal_info.write.jdbc(url=jdbc_url, table='employee_personal_info', mode=mode, properties=config)
review_id_df.write.jdbc(url= jdbc_url, table= 'review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [20]:
# write customer_df to RDS
customers_df.write.jdbc(url= jdbc_url, table= 'customers', mode=mode, properties= config)

Py4JJavaError: ignored